In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
np.random.seed(42)
tf.random.set_seed(42)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:10000].reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test[:2000].reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train[:10000])
y_test = to_categorical(y_test[:2000])
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    x_train, y_train,
    epochs=5,
    validation_split=0.1,
    verbose=1,
    batch_size=128  )
def generate_adversarial_examples(x, y, model, epsilon=0.1):
    adv_examples = []
    for i in range(len(x)):
        sample = tf.convert_to_tensor(x[i:i+1], dtype=tf.float32)
        label = tf.convert_to_tensor(y[i:i+1], dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(sample)
            predictions = model(sample)
            loss = tf.keras.losses.categorical_crossentropy(label, predictions)
        gradients = tape.gradient(loss, sample)
        signed_grad = tf.sign(gradients)
        perturbed_x = sample + epsilon * signed_grad
        perturbed_x = tf.clip_by_value(perturbed_x, 0, 1)
        adv_examples.append(perturbed_x.numpy())
    adv_examples = np.array(adv_examples)
    adv_examples = np.squeeze(adv_examples, axis=1)
    print(f"Shape of adversarial examples: {adv_examples.shape}")
    return adv_examples

x_adv = generate_adversarial_examples(x_test, y_test, model, epsilon=0.1)
assert x_adv.shape == (x_test.shape[0], 28, 28, 1), f"Shape mismatch: {x_adv.shape}"
adv_loss, adv_acc = model.evaluate(x_adv, y_test)
print(f"\nAdversarial Test Accuracy: {adv_acc:.4f}")
def tangent_distance(x1, x2):
    x1_flat = x1.flatten()
    x2_flat = x2.flatten()
    return np.linalg.norm(x1_flat - x2_flat)
sample_dist = tangent_distance(x_test[0], x_test[1])
print(f"\nTangent Distance between samples: {sample_dist:.4f}")

Epoch 1/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.5292 - loss: 1.5628 - val_accuracy: 0.8960 - val_loss: 0.3797
Epoch 2/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.8931 - loss: 0.3734 - val_accuracy: 0.9170 - val_loss: 0.2765
Epoch 3/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9195 - loss: 0.2793 - val_accuracy: 0.9230 - val_loss: 0.2401
Epoch 4/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.9322 - loss: 0.2334 - val_accuracy: 0.9290 - val_loss: 0.2157
Epoch 5/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9414 - loss: 0.1995 - val_accuracy: 0.9360 - val_loss: 0.1961
Shape of adversarial examples: (2000, 28, 28, 1)
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3690 - loss: 2.1864

Adversarial Test Accuracy: 0.3525

Tangent Distance between samples: 11.2688
